In [ ]:
# mount gdrive to access the dataset and weights
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# install and import libraries
!pip install ultralytics

from ultralytics import YOLO
from pathlib import Path
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

import cv2
import glob
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 11.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
# define path for best.pt, test dataset, and result save
custommodel_path = "/content/gdrive/My Drive/FYP/DATASETS_V2/version1/300epochs/runs/segment/train/weights/best.pt"
test_dataset_path = Path("/content/gdrive/My Drive/FYP/DATASETS_V2/version1/test/images")
results_save_path = Path("/content/gdrive/My Drive/FYP/DATASETS_V2/version1/300epochs/predictions")

# create the results directory if it does not exist
os.makedirs(results_save_path, exist_ok=True)

# get all .jpg files in the test dataset directory
image_paths = glob.glob(str(test_dataset_path / "*.jpg"))


In [ ]:
# load the YOLO model
model = YOLO(custommodel_path)

In [ ]:
# define colors for the classes
colors = {
    0: [0, 255, 255],   # yellow for overripe
    1: [0, 0, 255],     # red for ripe
    2: [128, 0, 128]    # purple for underripe
}

In [ ]:
# iterate over images and process them
for image_path in tqdm(image_paths, desc="Processing images"):
    # load the image using cv2
    image = cv2.imread(image_path)

    # check if image already uploaded
    if image is None:
        print(f"Failed to load image: {image_path}")
        continue

    # perform segmentation using YOLOv8
    results = model.predict(image)

    # draw the predictions masks on the image
    for result in results:
        if hasattr(result, 'masks') and result.masks is not None:
            for mask, cls in zip(result.masks.data, result.boxes.cls):
                mask = mask.cpu().numpy()

                # convert the mask to the same size as the image
                mask_resized = cv2.resize(mask, (image.shape[1], image.shape[0]))

                # insert color on prediction mask to identify classes
                color = colors.get(int(cls), [255, 255, 255])  # default to white if class is unknown
                image[mask_resized > 0] = color

    # extract the filename from the image path
    filename = os.path.basename(image_path)
    # Set the save path for the result
    save_path = results_save_path / filename

    # save the result to the specified path
    cv2.imwrite(str(save_path), image)
    print(f"Saved result to: {save_path}")

Processing images:   0%|          | 0/39 [00:00<?, ?it/s]


0: 640x640 1 ripe, 725.6ms
Speed: 16.0ms preprocess, 725.6ms inference, 33.6ms postprocess per image at shape (1, 3, 640, 640)


Processing images:   3%|▎         | 1/39 [00:05<03:21,  5.31s/it]

Saved result to: /content/gdrive/My Drive/FYP/DATASETS_V2/version1/300epochs/predictions/c9b4905b-64_jpg.rf.6c4b665a69c4809d49756018717d311d.jpg

0: 640x640 1 overripe, 283.6ms
Speed: 5.9ms preprocess, 283.6ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)


Processing images:   5%|▌         | 2/39 [00:06<01:37,  2.64s/it]

...

Saved result to: /content/gdrive/My Drive/FYP/DATASETS_V2/version1/300epochs/predictions/f171298e-43_jpg.rf.9d14018e3311da228f205df91138958b.jpg

0: 640x640 1 ripe, 287.5ms
Speed: 3.8ms preprocess, 287.5ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)


Processing images:  97%|█████████▋| 38/39 [00:53<00:01,  1.36s/it]

Saved result to: /content/gdrive/My Drive/FYP/DATASETS_V2/version1/300epochs/predictions/87159358-25_jpg.rf.ad81aad1dfc22f3b4e4555fd0c591e93.jpg

0: 640x640 1 underripe, 296.4ms
Speed: 3.4ms preprocess, 296.4ms inference, 5.3ms postprocess per image at shape (1, 3, 640, 640)


Processing images: 100%|██████████| 39/39 [00:54<00:00,  1.40s/it]

Saved result to: /content/gdrive/My Drive/FYP/DATASETS_V2/version1/300epochs/predictions/e86dfdee-80_jpg.rf.8f969f225657e34a26446c5197eaa54a.jpg
